In [2]:
import subprocess
from pydub import AudioSegment
import math


# Function to extract audio from video
def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


# Path to the video file
video_path = "./files/jocoding-gpt.mp4"
# Path to save the extracted audio file
audio_path = "./files/jocoding-gpt.mp3"

# Extract audio from the video file
extract_audio_from_video(video_path, audio_path)

# Load the extracted audio file
track = AudioSegment.from_mp3(audio_path)

# Define 10 minutes in milliseconds
ten_minutes = 10 * 60 * 1000

# Calculate the number of chunks
chunks = math.ceil(len(track) / ten_minutes)

# Export chunks of audio
for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes

    chunk = track[start_time:end_time]

    # Export each chunk as a separate mp3 file
    chunk.export(f"./files/chunks/chunk_{i}.mp3", format="mp3")